## Build Data

In [ ]:
filename = 'samples_zhihu_pro.jsonl'
lang = 'Chinese'

In [ ]:
import jsonlines
samples_pro = []
with jsonlines.open(filename) as reader:
    for item in reader:
        samples_pro.append(item)

In [ ]:
# print(samples_pro[0]['passage'])

In [ ]:
samples_pro = [sample for sample in samples_pro if 'memory_maze' in sample]
# samples_pro.pop(334)
# samples_pro.pop(334)
# samples_pro.pop(391)
len(samples_pro)

In [ ]:
import re
def cut_sent_cn(para):
    para = re.sub('([。！？\?])([^”’」])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’」])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’」])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’」])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")
# i = 5
# j = 1
# for k in samples_pro[i]['memory_maze']:
#     if k.startswith(f'sl{j}'):
#         for sent in cut_sent_cn(samples_pro[i]['memory_maze'][k]['content']):
#             print(sent)
#         print("*****")

In [ ]:
import sys
sys.path.append('..')
from agents.memory_tree import MemoryTree
from utils.file_utils import jsonl_filename_suffix
def build_lines_one(sample, end_idx=10):
    mem = MemoryTree(sample['memory_maze'])
    sents = mem.get_originals()
    # print(len(sents))
    summary, _ = mem.get_summary(end_idx=end_idx)
    # print(len(summary))
    ret = summary + sents[end_idx:end_idx+10-len(summary)]
    return ret
build_lines_one(samples_pro[2], 9)

In [ ]:
def build_sys(j):
    sys = "Style: \n{0}\n\nBackground: \n{1}\n\nPlot: \n{2}\n\nProtagonists: \n{3}".format(j['style'], j['background'], j['plot'], j['protagonists'])
    sys2 = 'We are writing this story block by block. Each block is around 200 words. Please follow the setting and style. The outline should be finished in around 10k words. '
    return {'role':'system', 'content': sys+'\n\n'+sys2}
print(build_sys(samples_pro[0])['content'])

In [ ]:
def lines2turns(lines, sys, lang='Chinese'):
    user1 = f'Write the next block of the story in {lang}. '
    user2 = lines[0].strip()
    # turns1 = [{'role': 'system', 'content': sys}]
    # turns2 = [{'role': 'system', 'content': sys}]
    turns1 = [{'role': 'user', 'content': sys+'\n'+user1}]
    turns2 = [{'role': 'user', 'content': sys+'\n'+user1+'\n'+user2}]
    for i in range(0, len(lines)-1, 2):
        turns1.append({'role': 'assistant', 'content': lines[i]})
        turns1.append({'role': 'user', 'content': lines[i+1]})
    for i in range(1, len(lines)-1, 2):
        turns2.append({'role': 'assistant', 'content': lines[i]})
        turns2.append({'role': 'user', 'content': lines[i+1]})
    return turns1, turns2
turns1, turns2 = lines2turns(build_lines_one(samples_pro[10], 40), build_sys(samples_pro[0])['content'], lang)
turns2

In [ ]:
messages_lst = []
import random
from tqdm import tqdm
i = 0
for sample in tqdm(samples_pro):
    i += 1
    mem = MemoryTree(sample['memory_maze'], lang=lang)
    end_idxs = random.sample(range(2, len(mem.get_originals())-1), 20)
    for end_idx in end_idxs:
        turns1, turns2 = lines2turns(build_lines_one(sample, end_idx), build_sys(sample)['content'], lang)
        messages_lst.append(turns1)
        messages_lst.append(turns2)

In [ ]:
print(len(messages_lst))
messages_lst[100]

In [ ]:
import json
def lst2jsonl(messages_lst, fn):
    with open(fn, "w") as f:
        for j in messages_lst:
            f.write(json.dumps({"messages": j}) + "\n")
lst2jsonl(messages_lst, jsonl_filename_suffix(filename, 'mem'))

In [ ]:
def deduplicate_jsonl(input_file, output_file):
    # Create a set to store unique lines of the JSONL file
    unique_lines = set()

    # Open the input file in read mode
    with open(input_file, 'r') as f:
        # Read each line from the input file
        for line in f:
            # Convert the line to a dictionary
            data = json.loads(line.strip())
            # Convert the dictionary back to a string
            json_string = json.dumps(data, sort_keys=True)
            # Add the string to the set
            unique_lines.add(json_string)

    # Open the output file in write mode
    with open(output_file, 'w') as f:
        # Write each unique line to the output file
        for line in unique_lines:
            f.write(line + '\n')

# Example usage
deduplicate_jsonl(jsonl_filename_suffix(filename, 'mem'), jsonl_filename_suffix(jsonl_filename_suffix(filename, 'mem'), 'deduplicated'))

In [ ]:
import random
def random_sample_half(input_file, output_file):
    # Read lines from input file
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Get half the length of lines
    half_length = len(lines) // 2

    # Randomly select half the lines
    sampled_lines = random.sample(lines, half_length)

    print(len(lines), len(sampled_lines))

    # Save sampled lines to output file
    with open(output_file, 'w') as outfile:
        outfile.writelines(sampled_lines)
random_sample_half(jsonl_filename_suffix(jsonl_filename_suffix(filename, 'mem'), 'deduplicated'), jsonl_filename_suffix(jsonl_filename_suffix(jsonl_filename_suffix(filename, 'mem'), 'deduplicated'), 'half'))